In [3]:
!pip install torch torchvision torchaudio
!pip install ogb
!pip install torch-geometric

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.3 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.3 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 25.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 

In [ ]:
import os, random, numpy as np, torch
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import to_networkx
import networkx as nx
import wandb

wandb.login(key="Your-api-key")
Dataset_name = "NCI1"

# Learnable Root-Based BFS Positional Encoding GCN

In [3]:
import os, random, numpy as np, torch
import torch.nn.functional as F
from torch import nn
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import to_undirected, degree
import wandb
from collections import deque

EPOCHS = int(os.getenv("EPOCHS", "50"))
NUM_SEEDS = int(os.getenv("NUM_SEEDS", "30"))
PROJECT = f"GCN Graph classification({Dataset_name})_Gal"
ENTITY = "noynetanel1-tel-aviv-university"

def set_seed(s):
    random.seed(s)
    np.random.seed(s)
    torch.manual_seed(s)
    torch.cuda.manual_seed_all(s)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def prep_graph(d):
    d = d.clone()
    if d.edge_index.numel() > 0:
        d.edge_index = to_undirected(d.edge_index, num_nodes=d.num_nodes)
    if d.x is None:
        d.x = torch.ones((d.num_nodes, 1), dtype=torch.float)
    deg = degree(d.edge_index[0], d.num_nodes, dtype=torch.float) if d.edge_index.numel() > 0 else torch.zeros(d.num_nodes, dtype=torch.float)
    m = max(deg.max().item(), 1.0)
    deg = (deg / m).unsqueeze(1)
    d.x = torch.cat([d.x, deg], dim=1)
    return d

def bfs_rank_from_root(num_nodes, edge_index_local, root_local):
    adj = [[] for _ in range(num_nodes)]
    u, v = edge_index_local
    for i in range(u.numel()):
        a = int(u[i])
        b = int(v[i])
        if a != b:
            adj[a].append(b)
            adj[b].append(a)
    ranks = torch.full((num_nodes,), -1.0)
    if num_nodes == 0 or root_local < 0 or root_local >= num_nodes:
        return ranks
    q = deque([root_local])
    seen = [False] * num_nodes
    seen[root_local] = True
    order = []
    while q:
        cur = q.popleft()
        order.append(cur)
        for w in adj[cur]:
            if not seen[w]:
                seen[w] = True
                q.append(w)
    L = max(1, len(order))
    for i, node in enumerate(order):
        ranks[node] = float(i) / float(L)
    return ranks

class LearnableBFS_GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, bfs_dim=8, tau=1.0):
        super().__init__()
        self.tau = tau
        self.root_scorer = nn.Sequential(
            nn.Linear(in_channels, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )
        self.bfs_proj = nn.Sequential(
            nn.Linear(1, 16),
            nn.ReLU(),
            nn.Linear(16, bfs_dim)
        )
        self.conv1 = GCNConv(in_channels + bfs_dim, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin = nn.Linear(hidden_channels, out_channels)

    @torch.no_grad()
    def _compute_bfs_rank_batch(self, x, edge_index, batch, root_logits):
        device = x.device
        N = x.size(0)
        ranks = torch.full((N,), -1.0, device='cpu')
        edge_index_cpu = edge_index.detach().cpu()
        batch_cpu = batch.detach().cpu()
        logits_cpu = root_logits.detach().cpu().squeeze(-1)
        num_graphs = int(batch_cpu.max().item()) + 1 if N > 0 else 0
        for g in range(num_graphs):
            idx = (batch_cpu == g).nonzero(as_tuple=False).view(-1)
            n = idx.numel()
            if n == 0:
                continue
            logits_g = logits_cpu[idx]
            probs_g = F.gumbel_softmax(logits_g, tau=self.tau, hard=True)
            root_local = int(probs_g.argmax().item())
            map_gl2lc = torch.full((N,), -1, dtype=torch.long)
            map_gl2lc[idx] = torch.arange(n, dtype=torch.long)
            u_all, v_all = edge_index_cpu
            m_u = map_gl2lc[u_all]
            m_v = map_gl2lc[v_all]
            mask = (m_u >= 0) & (m_v >= 0)
            u_loc = m_u[mask]
            v_loc = m_v[mask]
            ranks_g = bfs_rank_from_root(n, (u_loc, v_loc), root_local)
            ranks[idx] = ranks_g
        return ranks.to(device)

    def forward(self, x, edge_index, batch):
        root_logits = self.root_scorer(x)
        bfs_rank = self._compute_bfs_rank_batch(x, edge_index, batch, root_logits)
        bfs_feat = self.bfs_proj(bfs_rank.unsqueeze(1))
        x = torch.cat([x, bfs_feat], dim=1)
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        return F.log_softmax(self.lin(x), dim=1)

def run_seed(seed):
    set_seed(seed)
    dataset = TUDataset(root='data/TUDataset', name=Dataset_name, transform=prep_graph)
    if dataset.num_features == 0:
        for data in dataset:
            data.x = torch.ones((data.num_nodes, 1))
    dataset = dataset.shuffle()
    num_graphs = len(dataset)
    train_size = int(0.8 * num_graphs)
    train_dataset = dataset[:train_size]
    test_dataset = dataset[train_size:]
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    in_channels = dataset[0].x.size(1)
    model = LearnableBFS_GCN(in_channels, 64, dataset.num_classes, bfs_dim=8, tau=1.0).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    wandb.init(
        project=PROJECT,
        entity=ENTITY,
        name=f"GCN_LearnableBFS_STG_seed_{seed}",
        group=f"GCN_{Dataset_name}_LearnableBFS",
        job_type="learnable_bfs",
        reinit=True,
        config={
            "model": "GCN",
            "hidden_channels": 64,
            "epochs": EPOCHS,
            "lr": 0.001,
            "dataset": Dataset_name,
            "task": "Learnable BFS",
            "positional_encoding": "Learned root (ST-Gumbel) + BFS rank MLP",
            "seed": seed
        }
    )

    def train():
        model.train()
        total_loss = 0.0
        correct = 0
        total_samples = 0
        for data in train_loader:
            data = data.to(device)
            optimizer.zero_grad()
            out = model(data.x, data.edge_index, data.batch)
            loss = F.nll_loss(out, data.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
            total_samples += data.num_graphs
        train_acc = correct / total_samples
        return total_loss / len(train_loader), train_acc

    def test(loader):
        model.eval()
        correct = 0
        total_samples = 0
        for data in loader:
            data = data.to(device)
            out = model(data.x, data.edge_index, data.batch)
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
            total_samples += data.num_graphs
        return correct / total_samples

    for epoch in range(1, EPOCHS + 1):
        loss, train_acc = train()
        test_acc = test(test_loader)
        print(f"Seed {seed} | Epoch {epoch:03d} | Loss: {loss:.4f} | Train Acc: {train_acc:.4f} | Test Acc: {test_acc:.4f}")
        wandb.log({"epoch": epoch, "loss": loss, "train_acc": train_acc, "test_acc": test_acc})

    wandb.finish()

if __name__ == "__main__":
    wandb.login()
    for s in range(1, NUM_SEEDS + 1):
        run_seed(s)


Processing...
Done!
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250822_075100-gs9dv68o
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run GCN_LearnableBFS_STG_seed_1
wandb: ⭐️ View project at https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28NCI1%29_Gal
wandb: 🚀 View run at https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28NCI1%29_Gal/runs/gs9dv68o


Seed 1 | Epoch 001 | Loss: 0.6913 | Train Acc: 0.5210 | Test Acc: 0.5937
Seed 1 | Epoch 002 | Loss: 0.6784 | Train Acc: 0.5915 | Test Acc: 0.5998
Seed 1 | Epoch 003 | Loss: 0.6685 | Train Acc: 0.6058 | Test Acc: 0.6277
Seed 1 | Epoch 004 | Loss: 0.6601 | Train Acc: 0.6210 | Test Acc: 0.6046
Seed 1 | Epoch 005 | Loss: 0.6538 | Train Acc: 0.6229 | Test Acc: 0.6436
Seed 1 | Epoch 006 | Loss: 0.6395 | Train Acc: 0.6566 | Test Acc: 0.6448
Seed 1 | Epoch 007 | Loss: 0.6303 | Train Acc: 0.6569 | Test Acc: 0.6411
Seed 1 | Epoch 008 | Loss: 0.6252 | Train Acc: 0.6633 | Test Acc: 0.6521
Seed 1 | Epoch 009 | Loss: 0.6236 | Train Acc: 0.6554 | Test Acc: 0.6496
Seed 1 | Epoch 010 | Loss: 0.6148 | Train Acc: 0.6655 | Test Acc: 0.6411
Seed 1 | Epoch 011 | Loss: 0.6156 | Train Acc: 0.6740 | Test Acc: 0.6314
Seed 1 | Epoch 012 | Loss: 0.6151 | Train Acc: 0.6600 | Test Acc: 0.6436
Seed 1 | Epoch 013 | Loss: 0.6079 | Train Acc: 0.6718 | Test Acc: 0.6496
Seed 1 | Epoch 014 | Loss: 0.6100 | Train Acc: 0.67

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇███
wandb:      loss █▇▇▆▆▅▄▄▄▄▃▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▁▂▁▁▁
wandb:  test_acc ▁▁▃▂▅▄▅▅▄▄▅▄▄▄▅▆▅▄▆▆▇▆▅▇▇▆▆▇▆▇▇█▆▆▇██▇▇█
wandb: train_acc ▁▄▄▅▅▆▆▆▆▇▆▇▆▇▇▇▇▇▇█▇▇███████▇██████▇███
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.56676
wandb:  test_acc 0.69343
wandb: train_acc 0.70955
wandb: 
wandb: 🚀 View run GCN_LearnableBFS_STG_seed_1 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28NCI1%29_Gal/runs/gs9dv68o
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28NCI1%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250822_075100-gs9dv68o/logs
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/working/w

Seed 2 | Epoch 001 | Loss: 0.6918 | Train Acc: 0.5310 | Test Acc: 0.5243
Seed 2 | Epoch 002 | Loss: 0.6796 | Train Acc: 0.5821 | Test Acc: 0.6241
Seed 2 | Epoch 003 | Loss: 0.6686 | Train Acc: 0.6095 | Test Acc: 0.6131
Seed 2 | Epoch 004 | Loss: 0.6574 | Train Acc: 0.6341 | Test Acc: 0.6217
Seed 2 | Epoch 005 | Loss: 0.6514 | Train Acc: 0.6320 | Test Acc: 0.6302
Seed 2 | Epoch 006 | Loss: 0.6417 | Train Acc: 0.6436 | Test Acc: 0.6618
Seed 2 | Epoch 007 | Loss: 0.6319 | Train Acc: 0.6539 | Test Acc: 0.5985
Seed 2 | Epoch 008 | Loss: 0.6300 | Train Acc: 0.6545 | Test Acc: 0.6448
Seed 2 | Epoch 009 | Loss: 0.6265 | Train Acc: 0.6594 | Test Acc: 0.6180
Seed 2 | Epoch 010 | Loss: 0.6221 | Train Acc: 0.6639 | Test Acc: 0.6484
Seed 2 | Epoch 011 | Loss: 0.6190 | Train Acc: 0.6588 | Test Acc: 0.6533
Seed 2 | Epoch 012 | Loss: 0.6163 | Train Acc: 0.6651 | Test Acc: 0.6594
Seed 2 | Epoch 013 | Loss: 0.6114 | Train Acc: 0.6767 | Test Acc: 0.6533
Seed 2 | Epoch 014 | Loss: 0.6113 | Train Acc: 0.67

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇██
wandb:      loss █▇▆▆▅▄▄▄▄▃▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁
wandb:  test_acc ▁▅▅▅▆▆▅▆▇▇▅▆▇▇▆▆▇▇▆▇▇▇▇▇▇█▇▅▇█▇▄▇▇██▇███
wandb: train_acc ▁▃▄▅▅▆▆▆▆▆▆▆▆▇▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇█▇▇▇█▇█▇██
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.56798
wandb:  test_acc 0.67883
wandb: train_acc 0.71959
wandb: 
wandb: 🚀 View run GCN_LearnableBFS_STG_seed_2 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28NCI1%29_Gal/runs/k0d5dd18
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28NCI1%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250822_075742-k0d5dd18/logs
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/working/w

...

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
wandb:      loss █▇▆▆▆▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▁▂▂▂▁▁▁▁▁▁
wandb:  test_acc ▁▄▅▆▅▆▆▆▆▇▆▆▆▆▆▇▇▇▇▇▇▆▇▇▇▇▇▅█▇▇▇▆▇▇▇▇██▇
wandb: train_acc ▁▃▄▄▅▅▅▆▆▆▆▆▆▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇█████████▇▇
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.57421
wandb:  test_acc 0.67762
wandb: train_acc 0.70255
wandb: 
wandb: 🚀 View run GCN_LearnableBFS_STG_seed_28 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28NCI1%29_Gal/runs/00goz18r
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28NCI1%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250822_105151-00goz18r/logs
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/working/

Seed 29 | Epoch 001 | Loss: 0.6900 | Train Acc: 0.5301 | Test Acc: 0.5985
Seed 29 | Epoch 002 | Loss: 0.6812 | Train Acc: 0.5654 | Test Acc: 0.6144
Seed 29 | Epoch 003 | Loss: 0.6721 | Train Acc: 0.6001 | Test Acc: 0.6302
Seed 29 | Epoch 004 | Loss: 0.6655 | Train Acc: 0.6061 | Test Acc: 0.6229
Seed 29 | Epoch 005 | Loss: 0.6557 | Train Acc: 0.6186 | Test Acc: 0.6533
Seed 29 | Epoch 006 | Loss: 0.6478 | Train Acc: 0.6414 | Test Acc: 0.6046
Seed 29 | Epoch 007 | Loss: 0.6395 | Train Acc: 0.6420 | Test Acc: 0.6582
Seed 29 | Epoch 008 | Loss: 0.6310 | Train Acc: 0.6569 | Test Acc: 0.6667
Seed 29 | Epoch 009 | Loss: 0.6261 | Train Acc: 0.6533 | Test Acc: 0.6764
Seed 29 | Epoch 010 | Loss: 0.6191 | Train Acc: 0.6615 | Test Acc: 0.6655
Seed 29 | Epoch 011 | Loss: 0.6177 | Train Acc: 0.6627 | Test Acc: 0.6521
Seed 29 | Epoch 012 | Loss: 0.6180 | Train Acc: 0.6624 | Test Acc: 0.6253
Seed 29 | Epoch 013 | Loss: 0.6075 | Train Acc: 0.6813 | Test Acc: 0.6496
Seed 29 | Epoch 014 | Loss: 0.6057 | T

wandb: uploading output.log
wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
wandb:      loss ██▇▆▆▅▄▄▄▄▃▃▃▃▃▃▃▃▃▂▃▂▂▂▂▂▃▂▂▂▂▂▂▁▂▁▁▁▁▁
wandb:  test_acc ▁▂▃▃▅▅▅▅▄▃▆▅▅▆▅▄▅▂▆▇▇▇▇▄▆▆▇▇▆▅▇▅▆▇█▇█▄▇█
wandb: train_acc ▁▂▄▄▄▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇▇███▇██▇█████
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.56713
wandb:  test_acc 0.70195
wandb: train_acc 0.71259
wandb: 
wandb: 🚀 View run GCN_LearnableBFS_STG_seed_29 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28NCI1%29_Gal/runs/5v23z2p1
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28NCI1%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250822_105829-5v23z2p1/logs
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved

Seed 30 | Epoch 001 | Loss: 0.6920 | Train Acc: 0.5137 | Test Acc: 0.5182
Seed 30 | Epoch 002 | Loss: 0.6790 | Train Acc: 0.5687 | Test Acc: 0.6107
Seed 30 | Epoch 003 | Loss: 0.6692 | Train Acc: 0.5946 | Test Acc: 0.5973
Seed 30 | Epoch 004 | Loss: 0.6576 | Train Acc: 0.6369 | Test Acc: 0.5560
Seed 30 | Epoch 005 | Loss: 0.6536 | Train Acc: 0.6308 | Test Acc: 0.6350
Seed 30 | Epoch 006 | Loss: 0.6406 | Train Acc: 0.6542 | Test Acc: 0.6484
Seed 30 | Epoch 007 | Loss: 0.6364 | Train Acc: 0.6496 | Test Acc: 0.6496
Seed 30 | Epoch 008 | Loss: 0.6292 | Train Acc: 0.6609 | Test Acc: 0.6411
Seed 30 | Epoch 009 | Loss: 0.6229 | Train Acc: 0.6721 | Test Acc: 0.6168
Seed 30 | Epoch 010 | Loss: 0.6276 | Train Acc: 0.6597 | Test Acc: 0.6314
Seed 30 | Epoch 011 | Loss: 0.6183 | Train Acc: 0.6718 | Test Acc: 0.6156
Seed 30 | Epoch 012 | Loss: 0.6163 | Train Acc: 0.6691 | Test Acc: 0.6582
Seed 30 | Epoch 013 | Loss: 0.6106 | Train Acc: 0.6679 | Test Acc: 0.6813
Seed 30 | Epoch 014 | Loss: 0.6068 | T

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
wandb:      loss █▇▇▆▆▅▅▄▅▄▄▃▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▂▁▂▁▂▁▁▁▁▁
wandb:  test_acc ▁▄▄▂▅▅▅▄▅▄▇▆▅▆▇▆▆▇▇▆▇▆▇▇▆▇▇▇█▇██▇█▇▇██▇█
wandb: train_acc ▁▃▄▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇█▇█▇███████████
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.56283
wandb:  test_acc 0.71168
wandb: train_acc 0.72263
wandb: 
wandb: 🚀 View run GCN_LearnableBFS_STG_seed_30 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28NCI1%29_Gal/runs/91rjdbbs
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28NCI1%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250822_110532-91rjdbbs/logs


# **Layerwise BFS**

In [4]:
# **Layerwise BFS for Protein**
import os, random, numpy as np, torch
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import to_networkx
import networkx as nx
import wandb

EPOCHS = int(os.getenv("EPOCHS", "50"))
NUM_SEEDS = int(os.getenv("NUM_SEEDS", "30"))
PROJECT = f"GCN Graph classification({Dataset_name})_Gal"
ENTITY = "noynetanel1-tel-aviv-university"

def set_seed(s):
    random.seed(s)
    np.random.seed(s)
    torch.manual_seed(s)
    torch.cuda.manual_seed_all(s)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, bfs_dim=8):
        super().__init__()
        self.bfs_proj = torch.nn.Sequential(
            torch.nn.Linear(1, 16),
            torch.nn.ReLU(),
            torch.nn.Linear(16, bfs_dim)
        )
        self.conv1 = GCNConv(in_channels + bfs_dim, hidden_channels)
        self.conv2 = GCNConv(hidden_channels + bfs_dim, hidden_channels)
        self.lin = torch.nn.Linear(hidden_channels, out_channels)

    def forward(self, x, bfs_rank, edge_index, batch):
        bfs_feat = self.bfs_proj(bfs_rank.unsqueeze(1))
        x = torch.cat([x, bfs_feat], dim=1)
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        bfs_feat2 = self.bfs_proj(bfs_rank.unsqueeze(1))
        x = torch.cat([x, bfs_feat2], dim=1)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        return F.log_softmax(self.lin(x), dim=1)

def make_bfs_rank_transform():
    def transform(data):
        G = to_networkx(data, to_undirected=True)
        bfs_rank = torch.full((data.num_nodes,), -1.0, dtype=torch.float)
        if G.number_of_nodes() > 0:
            degrees = dict(G.degree())
            root = max(degrees, key=degrees.get) if degrees else 0
            bfs_order = list(nx.bfs_tree(G, root))
            L = max(1, len(bfs_order))
            for i, node in enumerate(bfs_order):
                bfs_rank[node] = i / L
        data.bfs_rank = bfs_rank
        # REDDIT-BINARY has no node features; use degree as feature or 1s
        if not hasattr(data, 'x') or data.x is None:
            deg = torch.tensor([d for _, d in G.degree()], dtype=torch.float).unsqueeze(1)
            data.x = deg
        return data
    return transform

def run_seed(SEED):
    set_seed(SEED)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    dataset = TUDataset(root='data/TUDataset', name=Dataset_name, transform=make_bfs_rank_transform())
    torch.manual_seed(SEED)
    dataset = dataset.shuffle()
    train_dataset = dataset[:len(dataset)//2]
    test_dataset = dataset[len(dataset)//2:]
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

    wandb.init(
        project=PROJECT,
        entity=ENTITY,
        name=f"GCN_Layerwise_BFS_{Dataset_name}_seed_{SEED}",
        group=f"GCN_{Dataset_name}_Layerwise_BFS",
        job_type="layerwise-bfs",
        reinit=True,
        config={
            "model": "GCN",
            "hidden_channels": 64,
            "epochs": EPOCHS,
            "lr": 0.001,
            "dataset": Dataset_name,
            "task": "Layerwise_BFS_Injection",
            "seed": SEED,
            "cuda_available": torch.cuda.is_available(),
            "gpu_name": torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU",
            "cuda_version": torch.version.cuda if torch.cuda.is_available() else None
        }
    )

    model = GCN(
        in_channels=dataset.num_features,
        hidden_channels=64,
        out_channels=dataset.num_classes
    ).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    def train():
        model.train()
        total_loss = 0.0
        correct = 0
        for data in train_loader:
            data = data.to(device)
            optimizer.zero_grad()
            out = model(data.x, data.bfs_rank, data.edge_index, data.batch)
            loss = F.nll_loss(out, data.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * data.num_graphs
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
        acc = correct / len(train_dataset)
        return total_loss / len(train_dataset), acc
    
    def test():
        model.eval()
        correct = 0
        for data in test_loader:
            data = data.to(device)
            out = model(data.x, data.bfs_rank, data.edge_index, data.batch)
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
        return correct / len(test_dataset)
    
    for epoch in range(1, EPOCHS + 1):
        loss, train_acc = train()
        test_acc = test()
        print(f"Seed {SEED} | Epoch {epoch:03d} | Loss: {loss:.4f} | Train Acc: {train_acc:.4f} | Test Acc: {test_acc:.4f}")
        wandb.log({"epoch": epoch, "loss": loss, "train_acc": train_acc, "test_acc": test_acc})
    wandb.finish()

if __name__ == "__main__":
    wandb.login()
    for s in range(1, NUM_SEEDS + 1):
        run_seed(s)


wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250822_111207-7zk3imus
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run GCN_Layerwise_BFS_NCI1_seed_1
wandb: ⭐️ View project at https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28NCI1%29_Gal
wandb: 🚀 View run at https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28NCI1%29_Gal/runs/7zk3imus


Seed 1 | Epoch 001 | Loss: 0.6916 | Train Acc: 0.5182 | Test Acc: 0.6112
Seed 1 | Epoch 002 | Loss: 0.6796 | Train Acc: 0.5713 | Test Acc: 0.5348
Seed 1 | Epoch 003 | Loss: 0.6768 | Train Acc: 0.5825 | Test Acc: 0.5922
Seed 1 | Epoch 004 | Loss: 0.6677 | Train Acc: 0.6024 | Test Acc: 0.5951
Seed 1 | Epoch 005 | Loss: 0.6590 | Train Acc: 0.6146 | Test Acc: 0.6019
Seed 1 | Epoch 006 | Loss: 0.6540 | Train Acc: 0.6282 | Test Acc: 0.6117
Seed 1 | Epoch 007 | Loss: 0.6484 | Train Acc: 0.6297 | Test Acc: 0.6248
Seed 1 | Epoch 008 | Loss: 0.6453 | Train Acc: 0.6273 | Test Acc: 0.6326
Seed 1 | Epoch 009 | Loss: 0.6380 | Train Acc: 0.6443 | Test Acc: 0.6389
Seed 1 | Epoch 010 | Loss: 0.6357 | Train Acc: 0.6482 | Test Acc: 0.6472
Seed 1 | Epoch 011 | Loss: 0.6277 | Train Acc: 0.6574 | Test Acc: 0.6521
Seed 1 | Epoch 012 | Loss: 0.6271 | Train Acc: 0.6633 | Test Acc: 0.6292
Seed 1 | Epoch 013 | Loss: 0.6228 | Train Acc: 0.6613 | Test Acc: 0.6691
Seed 1 | Epoch 014 | Loss: 0.6196 | Train Acc: 0.66

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
wandb:      loss █▇▇▇▆▆▅▅▅▄▄▄▄▃▅▄▃▃▃▃▂▃▃▂▂▂▂▂▂▂▂▂▁▃▁▂▂▁▁▁
wandb:  test_acc ▄▁▄▄▄▅▅▆▆▆▇▇▆▅▇▇▇▇▇▆▇▇▆▇▇█▇▆▇██▅█▇█▇▇██▇
wandb: train_acc ▁▃▃▄▄▅▅▅▆▆▆▆▆▆▆▇▆▇▆▇▇▇▇▇▇▇▇█▇▇▇▇▇▇▇▇▇▇▇█
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.56914
wandb:  test_acc 0.68175
wandb: train_acc 0.72068
wandb: 
wandb: 🚀 View run GCN_Layerwise_BFS_NCI1_seed_1 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28NCI1%29_Gal/runs/7zk3imus
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28NCI1%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250822_111207-7zk3imus/logs
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/working

Seed 2 | Epoch 001 | Loss: 0.6941 | Train Acc: 0.5134 | Test Acc: 0.5353
Seed 2 | Epoch 002 | Loss: 0.6844 | Train Acc: 0.5499 | Test Acc: 0.6083
Seed 2 | Epoch 003 | Loss: 0.6766 | Train Acc: 0.5791 | Test Acc: 0.5830
Seed 2 | Epoch 004 | Loss: 0.6719 | Train Acc: 0.5951 | Test Acc: 0.6161
Seed 2 | Epoch 005 | Loss: 0.6669 | Train Acc: 0.5903 | Test Acc: 0.6277
Seed 2 | Epoch 006 | Loss: 0.6572 | Train Acc: 0.6229 | Test Acc: 0.6277
Seed 2 | Epoch 007 | Loss: 0.6501 | Train Acc: 0.6380 | Test Acc: 0.5956
Seed 2 | Epoch 008 | Loss: 0.6415 | Train Acc: 0.6418 | Test Acc: 0.6399
Seed 2 | Epoch 009 | Loss: 0.6409 | Train Acc: 0.6268 | Test Acc: 0.6560
Seed 2 | Epoch 010 | Loss: 0.6296 | Train Acc: 0.6526 | Test Acc: 0.6594
Seed 2 | Epoch 011 | Loss: 0.6243 | Train Acc: 0.6550 | Test Acc: 0.6642
Seed 2 | Epoch 012 | Loss: 0.6188 | Train Acc: 0.6623 | Test Acc: 0.6414
Seed 2 | Epoch 013 | Loss: 0.6150 | Train Acc: 0.6740 | Test Acc: 0.6667
Seed 2 | Epoch 014 | Loss: 0.6100 | Train Acc: 0.67

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb:      loss ██▇▇▇▆▅▅▅▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▂▁▂▁▁▁▁▁▁
wandb:  test_acc ▁▄▃▄▅▄▅▆▆▆▇▆▇▅▇▇▇▇▇▅▇▇▆▆█▇▆██▇▇██▇▆▇█▆██
wandb: train_acc ▁▂▃▄▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇███▇█▇███████▇█
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.56622
wandb:  test_acc 0.70219
wandb: train_acc 0.71971
wandb: 
wandb: 🚀 View run GCN_Layerwise_BFS_NCI1_seed_2 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28NCI1%29_Gal/runs/r2v11m4l
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28NCI1%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250822_111521-r2v11m4l/logs
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/working

...

wandb: uploading history steps 38-45, console lines 38-45
wandb: 
wandb: uploading history steps 46-46, summary; updating run metadata
wandb: updating run metadata
wandb: updating run metadata; uploading history steps 47-49, summary, console lines 46-49
wandb: uploading history steps 47-49, summary, console lines 46-49
wandb: uploading data
wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
wandb:      loss ██▇▇▇▆▆▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▂▂▂▁▂▁▁▁▁▁▁▁▁
wandb:  test_acc ▄▁▅▅▄▆▆▆▆▇▇▆▆▆▇▇▇▇▇█████▇▇█▇▇▆▇█████████
wandb: train_acc ▁▂▂▄▅▅▆▅▆▆▆▆▆▆▇▇▆▇▇▇▇█▇▇██▇▇▇███████████
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.56856
wandb:  test_acc 0.691
wandb: train_acc 0.71873
wandb: 
wandb: 🚀 View run GCN_Layerwise_BFS_NCI1_seed_28 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28NCI1%29_Gal/runs/rwmk6y2x
wandb: ⭐️ View project

Seed 29 | Epoch 001 | Loss: 0.6886 | Train Acc: 0.5304 | Test Acc: 0.5401
Seed 29 | Epoch 002 | Loss: 0.6797 | Train Acc: 0.5552 | Test Acc: 0.5212
Seed 29 | Epoch 003 | Loss: 0.6738 | Train Acc: 0.5820 | Test Acc: 0.6224
Seed 29 | Epoch 004 | Loss: 0.6695 | Train Acc: 0.5825 | Test Acc: 0.6423
Seed 29 | Epoch 005 | Loss: 0.6639 | Train Acc: 0.6083 | Test Acc: 0.6501
Seed 29 | Epoch 006 | Loss: 0.6578 | Train Acc: 0.6039 | Test Acc: 0.5693
Seed 29 | Epoch 007 | Loss: 0.6521 | Train Acc: 0.6224 | Test Acc: 0.6336
Seed 29 | Epoch 008 | Loss: 0.6479 | Train Acc: 0.6229 | Test Acc: 0.6657
Seed 29 | Epoch 009 | Loss: 0.6413 | Train Acc: 0.6433 | Test Acc: 0.6662
Seed 29 | Epoch 010 | Loss: 0.6360 | Train Acc: 0.6418 | Test Acc: 0.6496
Seed 29 | Epoch 011 | Loss: 0.6353 | Train Acc: 0.6457 | Test Acc: 0.6798
Seed 29 | Epoch 012 | Loss: 0.6284 | Train Acc: 0.6526 | Test Acc: 0.6482
Seed 29 | Epoch 013 | Loss: 0.6239 | Train Acc: 0.6633 | Test Acc: 0.6745
Seed 29 | Epoch 014 | Loss: 0.6189 | T

wandb: uploading summary; updating run metadata
wandb: uploading summary; updating run metadata; uploading output.log; uploading wandb-summary.json
wandb: uploading summary; updating run metadata
wandb: updating run metadata
wandb: 
wandb: uploading history steps 30-49, summary, console lines 30-49
wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
wandb:      loss █▇▇▇▇▆▆▅▅▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▂▂▂▁▁▁▁▂▁
wandb:  test_acc ▂▁▅▅▆▅▆▆▆▇▆▇▇▇▇▇▇▆▇▆█▇█▇▇█▇████▇█▇███▇█▆
wandb: train_acc ▁▂▃▃▄▄▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇█▇▇▇▇███▇▇████▇█
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.57169
wandb:  test_acc 0.66959
wandb: train_acc 0.71679
wandb: 
wandb: 🚀 View run GCN_Layerwise_BFS_NCI1_seed_29 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28NCI1%29_Gal/runs/ebxomvn1
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-avi

Seed 30 | Epoch 001 | Loss: 0.6916 | Train Acc: 0.5328 | Test Acc: 0.5328
Seed 30 | Epoch 002 | Loss: 0.6814 | Train Acc: 0.5513 | Test Acc: 0.6039
Seed 30 | Epoch 003 | Loss: 0.6746 | Train Acc: 0.5640 | Test Acc: 0.5937
Seed 30 | Epoch 004 | Loss: 0.6651 | Train Acc: 0.5976 | Test Acc: 0.6024
Seed 30 | Epoch 005 | Loss: 0.6590 | Train Acc: 0.5927 | Test Acc: 0.6263
Seed 30 | Epoch 006 | Loss: 0.6500 | Train Acc: 0.6341 | Test Acc: 0.6316
Seed 30 | Epoch 007 | Loss: 0.6433 | Train Acc: 0.6433 | Test Acc: 0.6443
Seed 30 | Epoch 008 | Loss: 0.6395 | Train Acc: 0.6423 | Test Acc: 0.6482
Seed 30 | Epoch 009 | Loss: 0.6335 | Train Acc: 0.6438 | Test Acc: 0.6034
Seed 30 | Epoch 010 | Loss: 0.6371 | Train Acc: 0.6273 | Test Acc: 0.6676
Seed 30 | Epoch 011 | Loss: 0.6262 | Train Acc: 0.6613 | Test Acc: 0.6667
Seed 30 | Epoch 012 | Loss: 0.6277 | Train Acc: 0.6501 | Test Acc: 0.6491
Seed 30 | Epoch 013 | Loss: 0.6255 | Train Acc: 0.6584 | Test Acc: 0.6574
Seed 30 | Epoch 014 | Loss: 0.6263 | T

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
wandb:      loss █▇▇▆▆▄▅▄▄▄▄▃▃▃▃▃▃▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▂▁
wandb:  test_acc ▁▄▃▄▅▅▅▄▆▆▆▆▆▆▅▅▅▆▅▆▅▆▆▇▇▇▇▇▇▆█▇▆▆▇▇█▇██
wandb: train_acc ▁▂▂▄▃▅▅▅▅▆▆▆▆▆▆▆▆▆▆▇▆▆▇▇▇▇▇▇▇▇▇█▇▇▇████▇
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.58233
wandb:  test_acc 0.71144
wandb: train_acc 0.691
wandb: 
wandb: 🚀 View run GCN_Layerwise_BFS_NCI1_seed_30 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28NCI1%29_Gal/runs/v795phwi
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28NCI1%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250822_125527-v795phwi/logs


# **GCN Learnable BFS Sinusoidal**

In [6]:
import os, random, numpy as np, torch
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import to_networkx
import networkx as nx
import wandb
import math

EPOCHS = int(os.getenv("EPOCHS", "50"))
NUM_SEEDS = int(os.getenv("NUM_SEEDS", "30"))
PROJECT = f"GCN Graph classification({Dataset_name})_Gal"
ENTITY = "noynetanel1-tel-aviv-university"

def set_seed(s):
    random.seed(s)
    np.random.seed(s)
    torch.manual_seed(s)
    torch.cuda.manual_seed_all(s)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

class LearnableBFSPositionalEncoding(nn.Module):
    def __init__(self, dim, max_len=512):
        super().__init__()
        self.lin = nn.Linear(1, dim)
        pe = torch.zeros(max_len, dim)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, dim, 2).float() * (-math.log(10000.0) / dim))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer("pe", pe)
        self.max_len = max_len
    def forward(self, bfs_rank):
        idx = (bfs_rank.clamp(0, 1) * (self.max_len - 1)).long()
        return self.lin(bfs_rank.unsqueeze(1)) + self.pe[idx]

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, bfs_dim=32, max_len=512):
        super().__init__()
        self.bfs_enc = LearnableBFSPositionalEncoding(bfs_dim, max_len)
        self.conv1 = GCNConv(in_channels + bfs_dim, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin = nn.Linear(hidden_channels, out_channels)
    def forward(self, x, bfs_rank, edge_index, batch):
        x = torch.cat([x, self.bfs_enc(bfs_rank)], dim=1)
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        return F.log_softmax(self.lin(x), dim=1)

def make_bfs_rank_transform():
    def transform(data):
        G = to_networkx(data, to_undirected=True)
        bfs_rank = torch.full((data.num_nodes,), -1.0, dtype=torch.float)
        if G.number_of_nodes() > 0:
            deg = dict(G.degree())
            root = max(deg, key=deg.get) if len(deg) > 0 else 0
            order = list(nx.bfs_tree(G, root))
            L = max(1, len(order))
            for i, node in enumerate(order):
                bfs_rank[node] = i / L
        data.bfs_rank = bfs_rank
        return data
    return transform
    
def run_seed(SEED):
    set_seed(SEED)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    dataset = TUDataset(root="data/TUDataset", name=Dataset_name, transform=make_bfs_rank_transform())
    torch.manual_seed(SEED)
    dataset = dataset.shuffle()
    num_graphs = len(dataset)
    train_size = int(0.8 * num_graphs)  # 80% for train
    train_dataset = dataset[:train_size]
    test_dataset = dataset[train_size:]
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

    wandb.init(
        project=PROJECT,
        entity=ENTITY,
        name=f"GCN_LearnableSinBFS_seed_{SEED}",
        group=f"GCN_{Dataset_name}_LearnableSinBFS",
        job_type="learnable-sin-bfs",
        reinit=True,
        config={
            "model": "GCN",
            "hidden_channels": 64,
            "epochs": EPOCHS,
            "lr": 0.001,
            "dataset": Dataset_name,
            "task": "Learnable_Sinusoidal_BFS",
            "seed": SEED,
            "cuda_available": torch.cuda.is_available(),
            "gpu_name": torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU",
            "cuda_version": torch.version.cuda if torch.cuda.is_available() else None
        }
    )

    model = GCN(
        in_channels=dataset.num_features,
        hidden_channels=64,
        out_channels=dataset.num_classes,
        bfs_dim=32,
        max_len=512
    ).to(device)

    opt = torch.optim.Adam(model.parameters(), lr=0.001)

    def train():
        model.train()
        total_loss = 0.0
        correct = 0
        total = 0
        for data in train_loader:
            data = data.to(device)
            out = model(data.x, data.bfs_rank, data.edge_index, data.batch)
            loss = F.nll_loss(out, data.y)
            opt.zero_grad()
            loss.backward()
            opt.step()
            total_loss += loss.item() * data.num_graphs
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
            total += data.num_graphs
        return total_loss / len(train_dataset), correct / total

    def test():
        model.eval()
        correct = 0
        for data in test_loader:
            data = data.to(device)
            out = model(data.x, data.bfs_rank, data.edge_index, data.batch)
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
        return correct / len(test_dataset)

    for epoch in range(1, EPOCHS + 1):
        loss, train_acc = train()
        test_acc = test()
        print(f"Seed {SEED} | Epoch {epoch:03d} | Loss: {loss:.4f} | Train Acc: {train_acc:.4f} | Test Acc: {test_acc:.4f}")
        wandb.log({"epoch": epoch, "loss": loss, "train_acc": train_acc, "test_acc": test_acc})

    wandb.finish()

if __name__ == "__main__":
    wandb.login()
    for s in range(1, NUM_SEEDS + 1):
        run_seed(s)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


epoch,▁▅█
loss,█▆▁
test_acc,▁▂█
train_acc,▁▄█
epoch,3
loss,0.67495
test_acc,0.60827
train_acc,0.57391


Seed 1 | Epoch 001 | Loss: 0.6853 | Train Acc: 0.5228 | Test Acc: 0.5243
Seed 1 | Epoch 002 | Loss: 0.6818 | Train Acc: 0.5432 | Test Acc: 0.5353
Seed 1 | Epoch 003 | Loss: 0.6749 | Train Acc: 0.5739 | Test Acc: 0.6083
Seed 1 | Epoch 004 | Loss: 0.6678 | Train Acc: 0.5824 | Test Acc: 0.6533
Seed 1 | Epoch 005 | Loss: 0.6621 | Train Acc: 0.5949 | Test Acc: 0.6034
Seed 1 | Epoch 006 | Loss: 0.6493 | Train Acc: 0.6283 | Test Acc: 0.6071
Seed 1 | Epoch 007 | Loss: 0.6454 | Train Acc: 0.6363 | Test Acc: 0.6119
Seed 1 | Epoch 008 | Loss: 0.6340 | Train Acc: 0.6460 | Test Acc: 0.6277
Seed 1 | Epoch 009 | Loss: 0.6394 | Train Acc: 0.6314 | Test Acc: 0.6180
Seed 1 | Epoch 010 | Loss: 0.6305 | Train Acc: 0.6469 | Test Acc: 0.6582
Seed 1 | Epoch 011 | Loss: 0.6227 | Train Acc: 0.6499 | Test Acc: 0.6569
Seed 1 | Epoch 012 | Loss: 0.6217 | Train Acc: 0.6655 | Test Acc: 0.6460
Seed 1 | Epoch 013 | Loss: 0.6149 | Train Acc: 0.6648 | Test Acc: 0.6460
Seed 1 | Epoch 014 | Loss: 0.6124 | Train Acc: 0.66

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
loss,███▇▇▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▃▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁
test_acc,▁▁▄▆▄▅▅▆▆▆▇▅▅▆▇▆▆▇▇▇▇▇▇▇▇▆▆▇▇▇▇▇▇██▆████
train_acc,▁▂▃▃▃▅▅▄▅▅▆▆▆▆▆▆▆▆▆▆▇▇▆▇▆▇▇▇▇▇▇▇▇███████
epoch,50
loss,0.5151
test_acc,0.70073
train_acc,0.74361


Seed 2 | Epoch 001 | Loss: 0.6877 | Train Acc: 0.5213 | Test Acc: 0.5341
Seed 2 | Epoch 002 | Loss: 0.6807 | Train Acc: 0.5471 | Test Acc: 0.6290
Seed 2 | Epoch 003 | Loss: 0.6721 | Train Acc: 0.5918 | Test Acc: 0.5827
Seed 2 | Epoch 004 | Loss: 0.6590 | Train Acc: 0.6055 | Test Acc: 0.6071
Seed 2 | Epoch 005 | Loss: 0.6509 | Train Acc: 0.6198 | Test Acc: 0.5864
Seed 2 | Epoch 006 | Loss: 0.6431 | Train Acc: 0.6232 | Test Acc: 0.6192
Seed 2 | Epoch 007 | Loss: 0.6429 | Train Acc: 0.6277 | Test Acc: 0.6302
Seed 2 | Epoch 008 | Loss: 0.6334 | Train Acc: 0.6502 | Test Acc: 0.6448
Seed 2 | Epoch 009 | Loss: 0.6288 | Train Acc: 0.6414 | Test Acc: 0.6071
Seed 2 | Epoch 010 | Loss: 0.6234 | Train Acc: 0.6487 | Test Acc: 0.5876
Seed 2 | Epoch 011 | Loss: 0.6272 | Train Acc: 0.6472 | Test Acc: 0.6484
Seed 2 | Epoch 012 | Loss: 0.6133 | Train Acc: 0.6630 | Test Acc: 0.6387
Seed 2 | Epoch 013 | Loss: 0.6094 | Train Acc: 0.6651 | Test Acc: 0.6630
Seed 2 | Epoch 014 | Loss: 0.6051 | Train Acc: 0.66

...

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
loss,███▇▇▆▆▆▆▅▅▆▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▃▂▂▂▁▂▃▁▁▁▁
test_acc,▁▄▆▂▅▇▇█▄▆▇▇▇▇▇▇▇▆█▇██▇▆▇█▇▇▇███▇███████
train_acc,▁▂▄▄▄▅▅▅▅▆▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇█▇█████████
epoch,50
loss,0.50416
test_acc,0.67518
train_acc,0.75061


Seed 29 | Epoch 001 | Loss: 0.6842 | Train Acc: 0.5338 | Test Acc: 0.5791
Seed 29 | Epoch 002 | Loss: 0.6745 | Train Acc: 0.5690 | Test Acc: 0.6204
Seed 29 | Epoch 003 | Loss: 0.6659 | Train Acc: 0.5967 | Test Acc: 0.6022
Seed 29 | Epoch 004 | Loss: 0.6569 | Train Acc: 0.6153 | Test Acc: 0.6460
Seed 29 | Epoch 005 | Loss: 0.6545 | Train Acc: 0.6137 | Test Acc: 0.5754
Seed 29 | Epoch 006 | Loss: 0.6453 | Train Acc: 0.6305 | Test Acc: 0.6472
Seed 29 | Epoch 007 | Loss: 0.6394 | Train Acc: 0.6429 | Test Acc: 0.6326
Seed 29 | Epoch 008 | Loss: 0.6349 | Train Acc: 0.6426 | Test Acc: 0.6606
Seed 29 | Epoch 009 | Loss: 0.6322 | Train Acc: 0.6448 | Test Acc: 0.6594
Seed 29 | Epoch 010 | Loss: 0.6253 | Train Acc: 0.6551 | Test Acc: 0.6703
Seed 29 | Epoch 011 | Loss: 0.6229 | Train Acc: 0.6563 | Test Acc: 0.6180
Seed 29 | Epoch 012 | Loss: 0.6180 | Train Acc: 0.6609 | Test Acc: 0.6217
Seed 29 | Epoch 013 | Loss: 0.6127 | Train Acc: 0.6664 | Test Acc: 0.6679
Seed 29 | Epoch 014 | Loss: 0.6089 | T

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
loss,██▇▇▇▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▃▃▄▃▃▂▂▃▂▂▂▂▂▂▂▂▁▁▁
test_acc,▁▃▂▅▁▄▅▅▆▃▆▆▅▅▆▆▆▆▅▆▇▆▆▇▆▆▇▆▇▆▇▇▇▇▇▇█▇█▇
train_acc,▁▂▃▃▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇███████
epoch,50
loss,0.49608
test_acc,0.69465
train_acc,0.75882


Seed 30 | Epoch 001 | Loss: 0.6861 | Train Acc: 0.5338 | Test Acc: 0.5255
Seed 30 | Epoch 002 | Loss: 0.6780 | Train Acc: 0.5617 | Test Acc: 0.5316
Seed 30 | Epoch 003 | Loss: 0.6690 | Train Acc: 0.5928 | Test Acc: 0.5280
Seed 30 | Epoch 004 | Loss: 0.6587 | Train Acc: 0.5988 | Test Acc: 0.6314
Seed 30 | Epoch 005 | Loss: 0.6449 | Train Acc: 0.6329 | Test Acc: 0.5365
Seed 30 | Epoch 006 | Loss: 0.6472 | Train Acc: 0.6204 | Test Acc: 0.6703
Seed 30 | Epoch 007 | Loss: 0.6387 | Train Acc: 0.6335 | Test Acc: 0.6241
Seed 30 | Epoch 008 | Loss: 0.6356 | Train Acc: 0.6314 | Test Acc: 0.6606
Seed 30 | Epoch 009 | Loss: 0.6259 | Train Acc: 0.6539 | Test Acc: 0.6557
Seed 30 | Epoch 010 | Loss: 0.6222 | Train Acc: 0.6530 | Test Acc: 0.6509
Seed 30 | Epoch 011 | Loss: 0.6159 | Train Acc: 0.6615 | Test Acc: 0.6630
Seed 30 | Epoch 012 | Loss: 0.6158 | Train Acc: 0.6603 | Test Acc: 0.6667
Seed 30 | Epoch 013 | Loss: 0.6120 | Train Acc: 0.6697 | Test Acc: 0.6350
Seed 30 | Epoch 014 | Loss: 0.6245 | T

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
loss,██▇▇▇▆▆▆▅▅▆▅▅▅▅▄▄▄▄▄▄▃▃▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁▂▁
test_acc,▁▁▁▅▁▅▆▆▆▆▅▅▇▆▆▇▆▇▇▆▆▇█▇██▇▇▇████▇▆▇███▇
train_acc,▁▂▃▃▄▄▄▅▅▅▅▅▅▆▆▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇████▇█
epoch,50
loss,0.49595
test_acc,0.691
train_acc,0.75578


# **BFS and centrality**

In [7]:
import os, random, numpy as np, torch
import torch.nn.functional as F
from torch import nn
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import to_networkx
import networkx as nx
import wandb

EPOCHS = int(os.getenv("EPOCHS", "50"))
NUM_SEEDS = int(os.getenv("NUM_SEEDS", "30"))
PROJECT = f"GCN Graph classification({Dataset_name})_Gal"
ENTITY = "noynetanel1-tel-aviv-university"

def set_seed(s):
    random.seed(s)
    np.random.seed(s)
    torch.manual_seed(s)
    torch.cuda.manual_seed_all(s)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, cent_dim=16):
        super().__init__()
        self.cent_enc = nn.Linear(1, cent_dim)
        self.conv1 = GCNConv(in_channels + cent_dim, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin = nn.Linear(hidden_channels, out_channels)
    def forward(self, x, cent2, edge_index, batch):
        if x is None:
            x = torch.ones((cent2.shape[0], 1), device=cent2.device)
        x = torch.cat([x, self.cent_enc(cent2.unsqueeze(1))], dim=1)
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        return F.log_softmax(self.lin(x), dim=1)

def make_centrality_transform():
    def transform(data):
        G = to_networkx(data, to_undirected=True)
        n = data.num_nodes
        deg_cent = nx.degree_centrality(G) if G.number_of_nodes() > 0 else {}
        cent = torch.zeros((n,), dtype=torch.float)
        for u in range(n):
            cent[u] = float(deg_cent.get(u, 0.0))
        data.cent2 = cent
        if getattr(data, "x", None) is None:
            data.x = torch.ones((n, 1), dtype=torch.float)
        return data
    return transform

def run_seed(SEED):
    set_seed(SEED)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    dataset = TUDataset(root="data/TUDataset", name=Dataset_name, transform=make_centrality_transform())
    torch.manual_seed(SEED)
    dataset = dataset.shuffle()
    num_graphs = len(dataset)
    train_size = int(0.8 * num_graphs)
    train_dataset = dataset[:train_size]
    test_dataset = dataset[train_size:]
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

    wandb.init(
        project=PROJECT,
        entity=ENTITY,
        name=f"GCN_CentralityOnly_seed_{SEED}",
        group=f"GCN_{Dataset_name}_CentralityOnly",
        job_type="centrality-only",
        reinit=True,
        config={
            "model": "GCN",
            "hidden_channels": 64,
            "epochs": EPOCHS,
            "lr": 0.001,
            "dataset": Dataset_name,
            "task": "Centrality_Only",
            "seed": SEED,
            "cuda_available": torch.cuda.is_available(),
            "gpu_name": torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU",
            "cuda_version": torch.version.cuda if torch.cuda.is_available() else None
        }
    )

    model = GCN(
        in_channels=dataset.num_features if dataset.num_features else 1,
        hidden_channels=64,
        out_channels=dataset.num_classes,
        cent_dim=16
    ).to(device)

    opt = torch.optim.Adam(model.parameters(), lr=0.001)

    def train():
        model.train()
        total_loss = 0.0
        correct = 0
        total = 0
        for data in train_loader:
            data = data.to(device)
            out = model(data.x, data.cent2, data.edge_index, data.batch)
            loss = F.nll_loss(out, data.y)
            opt.zero_grad()
            loss.backward()
            opt.step()
            total_loss += loss.item() * data.num_graphs
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
            total += data.num_graphs
        return total_loss / len(train_dataset), correct / total

    def test():
        model.eval()
        correct = 0
        for data in test_loader:
            data = data.to(device)
            out = model(data.x, data.cent2, data.edge_index, data.batch)
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
        return correct / len(test_dataset)

    for epoch in range(1, EPOCHS + 1):
        loss, train_acc = train()
        test_acc = test()
        print(f"Seed {SEED} | Epoch {epoch:03d} | Loss: {loss:.4f} | Train Acc: {train_acc:.4f} | Test Acc: {test_acc:.4f}")
        wandb.log({"epoch": epoch, "loss": loss, "train_acc": train_acc, "test_acc": test_acc})

    wandb.finish()

if __name__ == "__main__":
    wandb.login()
    for s in range(1, NUM_SEEDS + 1):
        run_seed(s)

Seed 1 | Epoch 001 | Loss: 0.6855 | Train Acc: 0.5417 | Test Acc: 0.6192
Seed 1 | Epoch 002 | Loss: 0.6614 | Train Acc: 0.6128 | Test Acc: 0.5925
Seed 1 | Epoch 003 | Loss: 0.6495 | Train Acc: 0.6232 | Test Acc: 0.6594
Seed 1 | Epoch 004 | Loss: 0.6445 | Train Acc: 0.6265 | Test Acc: 0.6667
Seed 1 | Epoch 005 | Loss: 0.6382 | Train Acc: 0.6326 | Test Acc: 0.6144
Seed 1 | Epoch 006 | Loss: 0.6332 | Train Acc: 0.6353 | Test Acc: 0.6557
Seed 1 | Epoch 007 | Loss: 0.6300 | Train Acc: 0.6451 | Test Acc: 0.6290
Seed 1 | Epoch 008 | Loss: 0.6231 | Train Acc: 0.6451 | Test Acc: 0.6217
Seed 1 | Epoch 009 | Loss: 0.6215 | Train Acc: 0.6502 | Test Acc: 0.6825
Seed 1 | Epoch 010 | Loss: 0.6149 | Train Acc: 0.6582 | Test Acc: 0.6776
Seed 1 | Epoch 011 | Loss: 0.6075 | Train Acc: 0.6560 | Test Acc: 0.6752
Seed 1 | Epoch 012 | Loss: 0.6053 | Train Acc: 0.6633 | Test Acc: 0.6472
Seed 1 | Epoch 013 | Loss: 0.5997 | Train Acc: 0.6624 | Test Acc: 0.6460
Seed 1 | Epoch 014 | Loss: 0.5928 | Train Acc: 0.67

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
loss,█▆▆▅▅▅▅▄▄▄▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▂▁▁▂
test_acc,▃▁▅▅▂▃▃▆▆▆▄▇▆▆▇▄▇▇▇▆▇▇▇▄▇▅▇█▆█▇▇▇█▅██▇▅█
train_acc,▁▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇█▇▇█████████████
epoch,50
loss,0.56889
test_acc,0.70681
train_acc,0.70438


Seed 2 | Epoch 001 | Loss: 0.6886 | Train Acc: 0.5447 | Test Acc: 0.5438
Seed 2 | Epoch 002 | Loss: 0.6737 | Train Acc: 0.5882 | Test Acc: 0.6363
Seed 2 | Epoch 003 | Loss: 0.6550 | Train Acc: 0.6192 | Test Acc: 0.6545
Seed 2 | Epoch 004 | Loss: 0.6442 | Train Acc: 0.6375 | Test Acc: 0.6326
Seed 2 | Epoch 005 | Loss: 0.6379 | Train Acc: 0.6436 | Test Acc: 0.6521
Seed 2 | Epoch 006 | Loss: 0.6387 | Train Acc: 0.6341 | Test Acc: 0.6594
Seed 2 | Epoch 007 | Loss: 0.6310 | Train Acc: 0.6460 | Test Acc: 0.6655
Seed 2 | Epoch 008 | Loss: 0.6212 | Train Acc: 0.6496 | Test Acc: 0.6679
Seed 2 | Epoch 009 | Loss: 0.6135 | Train Acc: 0.6624 | Test Acc: 0.6533
Seed 2 | Epoch 010 | Loss: 0.6088 | Train Acc: 0.6618 | Test Acc: 0.6800
Seed 2 | Epoch 011 | Loss: 0.6008 | Train Acc: 0.6709 | Test Acc: 0.6582
Seed 2 | Epoch 012 | Loss: 0.6015 | Train Acc: 0.6655 | Test Acc: 0.6788
Seed 2 | Epoch 013 | Loss: 0.6051 | Train Acc: 0.6670 | Test Acc: 0.6569
Seed 2 | Epoch 014 | Loss: 0.5909 | Train Acc: 0.67

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▆▆▅▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁
test_acc,▁▅▆▅▆▆▆▆▇▆▆▆▆▇▆▇▆▇▇▇▇▇▇▇▆██▇▆▆█▇▇▆▇▇█▆▇█
train_acc,▁▃▅▅▅▅▆▆▆▆▆▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇█▇▇█████▇█████
epoch,50
loss,0.56418
test_acc,0.71046
train_acc,0.7056


Seed 3 | Epoch 001 | Loss: 0.6900 | Train Acc: 0.5176 | Test Acc: 0.5949
Seed 3 | Epoch 002 | Loss: 0.6685 | Train Acc: 0.5955 | Test Acc: 0.6131
Seed 3 | Epoch 003 | Loss: 0.6495 | Train Acc: 0.6189 | Test Acc: 0.6180
Seed 3 | Epoch 004 | Loss: 0.6428 | Train Acc: 0.6293 | Test Acc: 0.6363
Seed 3 | Epoch 005 | Loss: 0.6392 | Train Acc: 0.6353 | Test Acc: 0.6314
Seed 3 | Epoch 006 | Loss: 0.6347 | Train Acc: 0.6481 | Test Acc: 0.6046
Seed 3 | Epoch 007 | Loss: 0.6338 | Train Acc: 0.6493 | Test Acc: 0.6436
Seed 3 | Epoch 008 | Loss: 0.6250 | Train Acc: 0.6539 | Test Acc: 0.6533
Seed 3 | Epoch 009 | Loss: 0.6217 | Train Acc: 0.6542 | Test Acc: 0.6387
Seed 3 | Epoch 010 | Loss: 0.6227 | Train Acc: 0.6490 | Test Acc: 0.6630
Seed 3 | Epoch 011 | Loss: 0.6136 | Train Acc: 0.6493 | Test Acc: 0.6679
Seed 3 | Epoch 012 | Loss: 0.6065 | Train Acc: 0.6688 | Test Acc: 0.6630
Seed 3 | Epoch 013 | Loss: 0.6060 | Train Acc: 0.6682 | Test Acc: 0.6715
Seed 3 | Epoch 014 | Loss: 0.5965 | Train Acc: 0.66

...

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▆▅▅▅▄▃▃▃▂▂▃▂▂▂▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁
test_acc,▃▂▁▄▄▄▅▄▅▆▆▆▅▄▇▇▇▇▇▇▇▆▇▇▇█▇▇▇███▇█▇▆██▇▆
train_acc,▁▂▃▃▃▄▄▅▅▅▅▅▆▅▆▆▇▇▆▇▇▇▇▇▇█▇▇▇▇▇▇█▇▇▇████
epoch,50
loss,0.55969
test_acc,0.68248
train_acc,0.71107


Seed 29 | Epoch 001 | Loss: 0.6887 | Train Acc: 0.5347 | Test Acc: 0.5207
Seed 29 | Epoch 002 | Loss: 0.6719 | Train Acc: 0.5882 | Test Acc: 0.6119
Seed 29 | Epoch 003 | Loss: 0.6545 | Train Acc: 0.6101 | Test Acc: 0.5839
Seed 29 | Epoch 004 | Loss: 0.6464 | Train Acc: 0.6235 | Test Acc: 0.6496
Seed 29 | Epoch 005 | Loss: 0.6482 | Train Acc: 0.6268 | Test Acc: 0.6350
Seed 29 | Epoch 006 | Loss: 0.6391 | Train Acc: 0.6302 | Test Acc: 0.6338
Seed 29 | Epoch 007 | Loss: 0.6335 | Train Acc: 0.6408 | Test Acc: 0.6521
Seed 29 | Epoch 008 | Loss: 0.6313 | Train Acc: 0.6372 | Test Acc: 0.6472
Seed 29 | Epoch 009 | Loss: 0.6251 | Train Acc: 0.6533 | Test Acc: 0.6521
Seed 29 | Epoch 010 | Loss: 0.6242 | Train Acc: 0.6451 | Test Acc: 0.6448
Seed 29 | Epoch 011 | Loss: 0.6207 | Train Acc: 0.6432 | Test Acc: 0.6411
Seed 29 | Epoch 012 | Loss: 0.6119 | Train Acc: 0.6578 | Test Acc: 0.6715
Seed 29 | Epoch 013 | Loss: 0.6161 | Train Acc: 0.6530 | Test Acc: 0.6642
Seed 29 | Epoch 014 | Loss: 0.6056 | T

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▆▆▆▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁
test_acc,▁▄▃▅▅▆▆▆▅▇▇▆▆▆▆▆▇▆▇▇▇▇▇▇▇▇▆▇▇▇█▇▆█▇███▇█
train_acc,▁▃▄▅▅▅▅▆▅▅▆▆▆▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇███▇▇█▇██▇██
epoch,50
loss,0.56521
test_acc,0.70073
train_acc,0.71137


Seed 30 | Epoch 001 | Loss: 0.6875 | Train Acc: 0.5341 | Test Acc: 0.5766
Seed 30 | Epoch 002 | Loss: 0.6683 | Train Acc: 0.5931 | Test Acc: 0.6375
Seed 30 | Epoch 003 | Loss: 0.6540 | Train Acc: 0.6131 | Test Acc: 0.6521
Seed 30 | Epoch 004 | Loss: 0.6387 | Train Acc: 0.6460 | Test Acc: 0.6423
Seed 30 | Epoch 005 | Loss: 0.6283 | Train Acc: 0.6542 | Test Acc: 0.6460
Seed 30 | Epoch 006 | Loss: 0.6215 | Train Acc: 0.6557 | Test Acc: 0.6545
Seed 30 | Epoch 007 | Loss: 0.6148 | Train Acc: 0.6548 | Test Acc: 0.6582
Seed 30 | Epoch 008 | Loss: 0.6240 | Train Acc: 0.6423 | Test Acc: 0.6764
Seed 30 | Epoch 009 | Loss: 0.6050 | Train Acc: 0.6624 | Test Acc: 0.6569
Seed 30 | Epoch 010 | Loss: 0.6117 | Train Acc: 0.6724 | Test Acc: 0.6825
Seed 30 | Epoch 011 | Loss: 0.6043 | Train Acc: 0.6624 | Test Acc: 0.6849
Seed 30 | Epoch 012 | Loss: 0.5950 | Train Acc: 0.6752 | Test Acc: 0.6898
Seed 30 | Epoch 013 | Loss: 0.5925 | Train Acc: 0.6761 | Test Acc: 0.6898
Seed 30 | Epoch 014 | Loss: 0.5920 | T

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▆▅▅▄▃▄▃▃▃▃▂▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▂▁▁▁▁▂▁▁
test_acc,▁▄▅▄▅▅▆▅▆▇▆▇▆▇▆▇▇▇▆▇▇▆▇▇▆▇▇█▇▆█▇██▇▇▆▇█▇
train_acc,▁▄▅▆▆▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇█▇▇▇█▇███▇██▇▇█▇████
epoch,50
loss,0.56488
test_acc,0.70316
train_acc,0.70773
